# Reporte CVA

CVA (Credit Valuation Adjustment) es una técnica que permite calcular la diferencia entre el valor de un portofolio dado y el valor de un portafolio de renta fija, considerando las probabilidades de incumplimiento para cada unidad de tiempo, con el fin de encontrar el valor de mercado de los derivados incluyendo los riesgos de incumplimiento.

Un método que puede explotar las capacidades de Pyhton es el diseño de simulaciones para obtener la exposición futura esperada (EFE) a partir de simulaciones para cada unidad de tiempo. Se propone que la dinámica entre el precio del subyacente en cada unidad de tiempo respecto la anterior se modele a través de un movimiento browniano geométrico.

La fórmula que caracteriza la valuación es: 

$$CVA(T)\,=\,\int_{0}^{T} E^{Q} \left [ LGD \frac{B_{0}}{B_{t}} E(t) | t\,=\,\tau \right ] dPD(0,t)$$

Donde $T$ es la duración del derivado, $B_{t}$ es el valor de una unidad monetaria dada la tasa actual llevada al plazo $t$, $E(t)$ es la exposición al tiempo $t$, $LGD$ es la pérdida esperada dado incumplimiento  y $PD(0,t)$ es la probabilidad que se incumpla en los pagos en alguno de los primeros $t$ periodos de tiempo.

$LGD$ se puede entender en términos simplistas como el porcentaje de deuda que pudiera cubrirse con la venta de activos al tiempo $t$. 

Se puede usar tanto para valuación de derivados como para encontrar el valor de mercado de un portafolio respecto a uno libre de riesgo pero con probabilidad positiva de incumplimiento en los cupones.